In [1]:
from geopy.distance import geodesic
import pandas as pd
import folium

# --------------------------
# 지도 초기화
# --------------------------
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# --------------------------
# 1. 지하철 출퇴근 인원 + 주변 기업 수
# --------------------------
toprank_df = pd.read_csv("../data/subway_complexity_toprank_대표호선.csv", encoding='utf-8')
coord_df = pd.read_csv("../data/지하철_주소_위경도_통합_all.csv", encoding='euc-kr')
coord_df = coord_df.dropna(subset=['주소', '위도', '경도'])
coord_df = coord_df[coord_df['주소'].str.startswith('서울특별시', na=False)]
toprank_df['total_commute'] = toprank_df['morning'] + toprank_df['evening']
grouped_toprank = toprank_df.groupby('역명', as_index=False).agg({
    'morning': 'sum', 'evening': 'sum', 'total_commute': 'sum'
})
top30_df = grouped_toprank.sort_values(by='total_commute', ascending=False).head(30)
subway_df = pd.merge(top30_df, coord_df, on='역명', how='inner')

# 기업 위치 불러오기
company_df = pd.read_csv("../data/Data_saramin_with_coordinates.csv", encoding='utf-8')
company_df['위도'] = pd.to_numeric(company_df['위도'], errors='coerce')
company_df['경도'] = pd.to_numeric(company_df['경도'], errors='coerce')
company_df = company_df.dropna(subset=['위도', '경도'])

# 지하철 마커 + 600m 이내 기업 수 표시
for _, row in subway_df.iterrows():
    station_coord = (row['위도'], row['경도'])
    station_name = row['역명']
    morning = row['morning']
    evening = row['evening']
    total_commute = row['total_commute']
    radius_m = 600

    # 반경 내 기업 수 계산
    nearby_count = company_df.apply(
        lambda x: geodesic(station_coord, (x['위도'], x['경도'])).meters <= radius_m,
        axis=1
    ).sum()

    popup_text = f"""
    <b>{station_name}</b><br>
    출근: {morning:,}명<br>
    퇴근: {evening:,}명<br>
    합계: {total_commute:,}명<br>
    <b>기업 수 (600m 이내):</b> {nearby_count}개
    """

    folium.Circle(
        location=station_coord,
        radius=radius_m,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.1,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# --------------------------
# 2. 서울시 대학교 주소
# --------------------------
university_df = pd.read_csv("../data/Data_University_with_coordinates.csv", encoding='utf-8')
university_df = university_df[~university_df['위도'].astype(str).str.contains('좌표 추출 실패')]
university_df = university_df[~university_df['경도'].astype(str).str.contains('좌표 추출 실패')]
university_df['위도'] = pd.to_numeric(university_df['위도'], errors='coerce')
university_df['경도'] = pd.to_numeric(university_df['경도'], errors='coerce')
university_df = university_df.dropna(subset=['위도', '경도'])

for _, row in university_df.iterrows():
    lat = row['위도']
    lon = row['경도']
    university_name = row['대학명']

    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(f"<대학교> {university_name}", max_width=300),
        icon=folium.Icon(color='blue', icon='university', prefix='fa')
    ).add_to(m)

# --------------------------
# 3. 서울시 기차역 (지하철 제외)
# --------------------------
train_df = pd.read_csv("../data/seoul_train_stations_coordinates.csv", encoding='utf-8')
train_df['위도'] = pd.to_numeric(train_df['위도'], errors='coerce')
train_df['경도'] = pd.to_numeric(train_df['경도'], errors='coerce')
train_df = train_df.dropna(subset=['위도', '경도'])

for _, row in train_df.iterrows():
    lat = row['위도']
    lon = row['경도']
    train_name = row['역이름']  # 열 이름이 다르면 '역명' 등으로 수정 필요

    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(f"<기차역> {train_name}", max_width=300),
        icon=folium.Icon(color='orange', icon='train', prefix='fa')
    ).add_to(m)

# --------------------------
# 4. 서울시 고속버스터미널 주소
# --------------------------
bus_df = pd.read_csv("../data/seoul_bus_stations_coordinates.csv", encoding='utf-8')
bus_df['위도'] = pd.to_numeric(bus_df['위도'], errors='coerce')
bus_df['경도'] = pd.to_numeric(bus_df['경도'], errors='coerce')
bus_df = bus_df.dropna(subset=['위도', '경도'])

for _, row in bus_df.iterrows():
    lat = row['위도']
    lon = row['경도']
    busStation_name = row['정거장이름']  # 실제 열 이름 확인 필요

    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(f"<버스터미널> {busStation_name}", max_width=300),
        icon=folium.Icon(color='cadetblue', icon='bus', prefix='fa')
    ).add_to(m)

# --------------------------
# 최종 저장
# --------------------------
output_path = "../data/일괄데이터_Folium화.html"
m.save(output_path)
print(f"✅ 통합 지도 저장 완료: {output_path}")


✅ 통합 지도 저장 완료: ../data/일괄데이터_Folium화.html
